In [10]:
from utils import profiler, reader
from typing import List, Dict, Tuple, Set
from tqdm import tqdm
from IPython.display import clear_output
import time

In [11]:
import numpy as np

In [12]:
datafile = "../data/day10_input.txt"
data = reader.read_from_file(datafile)
data = [list(map(int, list(x.rstrip()))) for x in data]
print(data)

[[9, 8, 1, 0, 5, 0, 1, 1, 2, 2, 1, 0, 1, 2, 1, 0, 3, 4, 5, 6, 5, 4, 3, 2, 3, 2, 1, 0, 3, 2, 1, 0, 3, 3, 2, 1, 2, 3, 4, 5, 6, 7, 5, 6, 7], [8, 7, 2, 3, 4, 1, 2, 0, 1, 0, 1, 4, 4, 3, 4, 9, 2, 1, 8, 7, 8, 8, 9, 1, 4, 3, 4, 5, 4, 9, 0, 1, 2, 3, 1, 0, 4, 3, 2, 1, 9, 8, 6, 7, 8], [7, 6, 5, 4, 3, 4, 3, 1, 8, 1, 2, 3, 5, 6, 7, 8, 9, 0, 9, 8, 9, 7, 6, 0, 0, 2, 3, 6, 7, 8, 5, 4, 3, 9, 8, 6, 5, 0, 1, 0, 2, 1, 5, 4, 9], [5, 4, 5, 9, 8, 5, 3, 2, 9, 8, 5, 4, 3, 5, 6, 7, 9, 2, 1, 0, 5, 6, 5, 4, 1, 1, 6, 7, 0, 7, 6, 2, 1, 6, 7, 7, 8, 9, 3, 4, 3, 0, 9, 3, 2], [4, 5, 6, 5, 7, 6, 4, 1, 6, 7, 6, 3, 4, 4, 7, 8, 8, 3, 4, 9, 8, 7, 4, 3, 2, 0, 9, 8, 1, 1, 0, 3, 0, 5, 4, 3, 0, 8, 2, 5, 6, 7, 8, 0, 1], [3, 2, 7, 6, 7, 6, 5, 0, 9, 8, 2, 3, 4, 3, 0, 9, 7, 6, 5, 4, 1, 0, 5, 0, 1, 2, 4, 3, 2, 2, 3, 4, 5, 8, 9, 2, 1, 2, 1, 0, 7, 8, 9, 9, 8], [2, 1, 8, 4, 8, 9, 6, 7, 8, 0, 1, 6, 5, 2, 1, 0, 5, 0, 1, 3, 2, 0, 1, 1, 2, 6, 7, 4, 0, 1, 2, 7, 6, 7, 3, 4, 2, 8, 9, 0, 1, 0, 8, 6, 7], [1, 0, 9, 3, 4, 5, 9, 0, 8, 9, 8, 7, 2, 

In [13]:
np.array(data)

array([[9, 8, 1, ..., 5, 6, 7],
       [8, 7, 2, ..., 6, 7, 8],
       [7, 6, 5, ..., 5, 4, 9],
       ...,
       [9, 6, 5, ..., 4, 3, 2],
       [0, 1, 2, ..., 5, 8, 1],
       [1, 0, 3, ..., 8, 9, 0]], shape=(45, 45))

In [25]:
example = ['78121874',
            '89010123',
            '87430965',
            '96549874',
            '45678903',
            '32019012',
            '01329801',
            '10456732']

example = [
'89010123',
'78121874',
'87430965',
'96549874',
'45678903',
'32019012',
'01329801',
'10456732'
]

example = [list(map(int, list(x))) for x in example]
example

[[8, 9, 0, 1, 0, 1, 2, 3],
 [7, 8, 1, 2, 1, 8, 7, 4],
 [8, 7, 4, 3, 0, 9, 6, 5],
 [9, 6, 5, 4, 9, 8, 7, 4],
 [4, 5, 6, 7, 8, 9, 0, 3],
 [3, 2, 0, 1, 9, 0, 1, 2],
 [0, 1, 3, 2, 9, 8, 0, 1],
 [1, 0, 4, 5, 6, 7, 3, 2]]

# Part 1

### Overview

We want to find all paths from a 0 to a 9 on the input grid. We need to measure distinct start and end points!

### Approach

We can depth first search all of the points that have a zero. When we reach a 9 we add one to the total.
We'll track the paths so that we can avoid overcounting!


In [40]:

@profiler.profile
def part1(grid: List[List[int]]) -> Tuple[List[List[Tuple[int, int]]], int]:
    """
    Finds all valid paths from 0 to 9 in a grid using Depth-First Search.

    Args:
        grid: A 2D list (m x n) of single-digit integers.

    Returns:
        A tuple: (list of paths, count of paths)
    """

    rows = len(grid)
    cols = len(grid[0])
    all_paths = []

    def get_neighbors(row, col, current_val):
        """Gets valid neighbors for a given cell."""
        neighbors = []
        possible_moves = [(0, 1), (0, -1), (1, 0), (-1, 0)]  # Right, left, down, up

        for dr, dc in possible_moves:
            new_row, new_col = row + dr, col + dc
            if 0 <= new_row < rows and 0 <= new_col < cols and grid[new_row][new_col] == current_val + 1:
                neighbors.append((new_row, new_col))
        return neighbors

    def dfs(row, col, path):
        """Performs Depth-First Search and collects all paths."""
        if grid[row][col] == 9:
            all_paths.append(path + [(row, col)]) # found a path, append it to the list
            return

        neighbors = get_neighbors(row, col, grid[row][col])
        for next_row, next_col in neighbors:
            dfs(next_row, next_col, path + [(row, col)])

    # Find the starting points (0)
    start_points = []
    for r in range(rows):
        for c in range(cols):
            if grid[r][c] == 0:
                start_points.append((r, c))

    for start_r, start_c in start_points:
        dfs(start_r, start_c, [])

    return all_paths, len(all_paths)

paths, count = part1(data)
scores = set()
for path in paths:
    scores.add((path[0], path[-1]))
print(len(scores), count)

Calling part1: Memory used 557056 kB; Execution Time: 0.0073143749996233964 s
552 1225


# Part 2

### Overview 

Now we need to measure the total number of paths.

### Approach

It turns out we solved this already in the previous part, so I'll just print out the answer!